In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
import heapq
from math import pow
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from collections import OrderedDict

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

## Encryption

In [ ]:
#keygen
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_GCM)
nonce = cipher.nonce
#encrypt
ciphertext, tag = cipher.encrypt_and_digest(b"fish")
encode_this = nonce + ciphertext + tag

## Sample until hash function matches

In [ ]:
s = ""
for b in encode_this:
    print("The text currently is :", s)
    k = input("Finish the current sentence.")
    s += " " + first_sentence(k)
    s = encode_byte(s,b)

In [ ]:
inputs = tokenizer("Richard Lewis is a", return_tensors="pt")
probs = torch.topk(nn.functional.softmax(model(**inputs).logits[:, -1, :], dim=-1),8)

In [ ]:
def leq_set(k,l):
    result = set([frozenset()])
    for i in range(k):
        new_result = result.copy()
        #print(new_result)
        for r in result:
            for j in l:
                new_r = r.union(frozenset([j]))
                #print(new_r)
                new_result.add(new_r)
        result = new_result

    return result


def set_combinations(l):
    result = set()
    l1 = leq_set(int(len(l)/2), l)
    l1.remove(frozenset())
    for i in l1:
        ln = [j for j in l if not j in i]
        l2 = leq_set(len(ln), ln)
        l2.remove(frozenset())
        for j in l2:
            t = (i,j)
            result.add(t)
    return result

In [ ]:
def balance_probs(probs):
    length = probs.values.size()[1]
    combs = set_combinations(range(length))
    indeces = None
    difference = 1
    for (comb1,comb2) in combs:
        sum1 = 0
        for i in comb1:
            sum1 += probs.values[0,i]
        sum2 = 0
        for i in comb2:
            sum2 += probs.values[0,i]

        a = torch.abs(sum1-sum2)
        if a < difference:
            difference = a
            indeces = (comb1,comb2)
        
            print(comb1, comb2)
            print(float(difference))
    return indeces

In [ ]:
top_k_top_p_filtering(something,top_k=1000)

## Decryption

In [ ]:
#decrypt
cipher = AES.new(key, AES.MODE_GCM,nonce)
cipher.decrypt(ciphertext)